In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
from sklearn import tree
from sklearn import ensemble
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error

# Ex. 7

In [6]:
dataset_file = '../datasets/Boston.csv'
seed = 1

data = pd.read_csv(dataset_file)
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [7]:
train_data, test_data = train_test_split(data, test_size=.25, random_state=seed)

train_X, test_X = train_data.drop(['medv'], axis=1), test_data.drop(['medv'], axis=1)
train_y, test_y = train_data.medv, test_data.medv

In [33]:
from multiprocessing import Pool
import os

In [32]:
# %%time
trees_nums = np.arange(1, 101)
vars_nums = np.array([1, 3, 5, 7, 9, 11, 13])
errors = pd.DataFrame(index=trees_nums, columns=vars_nums)

models = []
for vars_num in vars_nums:
    for trees_num in trees_nums:
        model = ensemble.RandomForestRegressor(
            random_state=seed,
            n_estimators=trees_num,
            max_features=vars_num
        )
        models.append(model)

def fit(model):
    model.fit(train_X, train_y)
    return model

pool = Pool(os.cpu_count() - 1)
fitted = pool.map(fit, models)
pool.close()
# pool.join()
    
for model in fitted:
    pred_y = model.predict(test_X)
    errors.loc[model.n_estimators, model.max_features] = mean_squared_error(test_y, pred_y)

NameError: name 'os' is not defined

In [28]:
%%time
trees_nums = np.arange(1, 101)
vars_nums = np.array([1, 3, 5, 7, 9, 11, 13])
errors = pd.DataFrame(index=trees_nums, columns=vars_nums)

for vars_num in vars_nums:
    for trees_num in trees_nums:
        model = ensemble.RandomForestRegressor(
            random_state=seed,
            n_estimators=trees_num,
            max_features=vars_num
        )
        model.fit(train_X, train_y)
        pred_y = model.predict(test_X)

        errors.loc[trees_num, vars_num] = mean_squared_error(test_y, pred_y)

CPU times: user 1min 4s, sys: 151 ms, total: 1min 4s
Wall time: 1min 4s


In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
for col in errors.columns:
    ax.plot(errors.index, errors[col], label=f'max features: {col}')
ax.legend()